In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [5]:
#Variable 
variable85="rcp85_prcptot_annual"
variable45="rcp45_prcptot_annual"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_prcptot_annual.nc']

# FOR RCP 45

## Subsetting arrays by periods of time and percentiles with Xclim

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [7]:
# Mapping data - 30 year means
tmp145 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time145 = tmp145.time[0::window]

ds30yavg45 = tmp145.coarsen(time=window).mean()
ds30yavg45['time'] = time145
perc30yavg45 = ens.ensemble_percentiles(ds30yavg45)
perc30yavg45 # entire grid

# Mapping data

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    prcptot_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    prcptot_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [8]:
%time df45 = perc30yavg45.drop('realization').to_dataframe().dropna()

df45

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 3min 32s, sys: 23min 46s, total: 27min 19s
Wall time: 37 s


prcptot_p10  prcptot_p50  prcptot_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01   242.776947   248.862839   253.129547
                     2011-01-01   250.770477   258.080261   273.385803
                     2041-01-01   268.117340   281.025696   298.878113
                     2071-01-01   284.791718   298.072601   314.719666
          -88.961884 1981-01-01   240.618851   246.484695   250.704636
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01  1217.491821  1282.380371  1355.752563
          -74.045807 1981-01-01  1175.379761  1226.125488  1238.832764
                     2011-01-01  1189.914307  1228.283569  1276.126465
                     2041-01-01  1209.976929  1288.678589  1329.441040
                     2071-01-01  1214.395996  1276.816772  1354.748291

[304824 rows x 3 columns]

## Round

In [9]:
df45C = df45.copy()
df45C["prcptot_p10"] = round(df45C["prcptot_p10"],2)
df45C["prcptot_p10"] = round(df45C["prcptot_p10"],2)
df45C["prcptot_p10"] = round(df45C["prcptot_p10"],2)

df45C

prcptot_p10  prcptot_p50  prcptot_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01       242.78   248.862839   253.129547
                     2011-01-01       250.77   258.080261   273.385803
                     2041-01-01       268.12   281.025696   298.878113
                     2071-01-01       284.79   298.072601   314.719666
          -88.961884 1981-01-01       240.62   246.484695   250.704636
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01      1217.49  1282.380371  1355.752563
          -74.045807 1981-01-01      1175.38  1226.125488  1238.832764
                     2011-01-01      1189.91  1228.283569  1276.126465
                     2041-01-01      1209.98  1288.678589  1329.441040
                     2071-01-01      1214.40  1276.816772  1354.748291

[304824 rows x 3 columns]

## Eliminate 2011, Pivot Table and Create new columns 

In [10]:
Region1i45 = df45C.reset_index()
print (Region1i45)
Region1i245 = Region1i45[Region1i45.time.dt.year!= 2011]
print ("2011 eliminated")
df45f = Region1i245.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df45f.columns = [year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df45f

              lat        lon       time  prcptot_p10  prcptot_p50  prcptot_p90
0       66.623306 -89.045212 1981-01-01       242.78   248.862839   253.129547
1       66.623306 -89.045212 2011-01-01       250.77   258.080261   273.385803
2       66.623306 -89.045212 2041-01-01       268.12   281.025696   298.878113
3       66.623306 -89.045212 2071-01-01       284.79   298.072601   314.719666
4       66.623306 -88.961884 1981-01-01       240.62   246.484695   250.704636
...           ...        ...        ...          ...          ...          ...
304819  40.041039 -74.129143 2071-01-01      1217.49  1282.380371  1355.752563
304820  40.041039 -74.045807 1981-01-01      1175.38  1226.125488  1238.832764
304821  40.041039 -74.045807 2011-01-01      1189.91  1228.283569  1276.126465
304822  40.041039 -74.045807 2041-01-01      1209.98  1288.678589  1329.441040
304823  40.041039 -74.045807 2071-01-01      1214.40  1276.816772  1354.748291

[304824 rows x 6 columns]
2011 eliminated
DF pivote

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212          973.83          1005.69          1012.57   
          -88.961884          982.36          1014.96          1021.90   
          -88.878548          974.80          1008.30          1013.75   
          -88.795219          970.07          1003.25          1007.35   
          -88.711891          970.61          1005.58          1012.53   
...                              ...              ...              ...   
66.623306 -61.629639          497.29           537.15           538.33   
          -61.546310          507.19           547.88           549.54   
          -61.462978          500.28           541.19           542.73   
          -61.379650          496.54           537.15           538.22   
          -61.296322          468.49           506.54           506.88   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212     1002.402344      1039.936279      1032.830688   
          -88.961884     1010.011169      1048.570557      1043.749878   
          -88.878548     1001.046082      1040.501587      1035.856567   
          -88.795219      993.866211      1037.199707      1028.240967   
          -88.711891      995.498840      1040.649048      1029.445435   
...                              ...              ...              ...   
66.623306 -61.629639      502.441711       551.150146       564.163818   
          -61.546310      512.102295       561.671448       574.705933   
          -61.462978      505.357391       555.124084       567.502686   
          -61.379650      501.231445       551.254639       564.786560   
          -61.296322      472.411102       520.979736       533.535217   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  
lat       lon                                                           
40.041039 -89.045212     1022.678040      1117.536621      1113.804077  
          -88.961884     1031.998901      1128.370361      1121.904419  
          -88.878548     1024.193115      1120.681030      1113.541504  
          -88.795219     1018.400696      1113.942505      1106.774780  
          -88.711891     1021.307312      1116.597168      1106.486206  
...                              ...              ...              ...  
66.623306 -61.629639      518.333801       592.527039       630.359619  
          -61.546310      528.213501       603.849792       642.918396  
          -61.462978      521.916626       595.150452       635.752258  
          -61.379650      517.425415       590.644897       630.589417  
          -61.296322      488.297882       556.210083       595.162781  

[76206 rows x 9 columns]

# FOR RCP 85

## Subsetting arrays by periods of time and percentiles with Xclim

In [11]:
dsEns85= ens.create_ensemble(files85)
dsEns85

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [12]:
# Mapping data - 30 year means
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time185 = tmp185.time[0::window]

ds30yavg85 = tmp185.coarsen(time=window).mean()
ds30yavg85['time'] = time185
perc30yavg85 = ens.ensemble_percentiles(ds30yavg85)
perc30yavg85 # entire grid

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    prcptot_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    prcptot_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [13]:
%time df85 = perc30yavg85.drop('realization').to_dataframe().dropna()
df85

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 1min 39s, sys: 12min 50s, total: 14min 29s
Wall time: 15.6 s


prcptot_p10  prcptot_p50  prcptot_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01   245.039719   248.596481   258.564270
                     2011-01-01   260.379547   266.665070   274.814545
                     2041-01-01   276.355499   296.327087   314.078552
                     2071-01-01   296.454498   320.437500   370.024689
          -88.961884 1981-01-01   242.724014   246.173447   256.140045
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01  1252.603271  1315.043823  1396.519409
          -74.045807 1981-01-01  1189.647095  1231.080933  1246.812256
                     2011-01-01  1202.679321  1257.817017  1306.370361
                     2041-01-01  1213.784058  1263.579712  1321.775024
                     2071-01-01  1245.081543  1309.232788  1389.138306

[304824 rows x 3 columns]

## Transform in Celsius and round

In [14]:
df85C = df85.copy()
df85C["prcptot_p10"] = round(df85C["prcptot_p10"],2)
df85C["prcptot_p50"] = round(df85C["prcptot_p50"],2)
df85C["prcptot_p90"] = round(df85C["prcptot_p90"],2)

df85C

prcptot_p10  prcptot_p50  prcptot_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01       245.04       248.60       258.56
                     2011-01-01       260.38       266.67       274.81
                     2041-01-01       276.36       296.33       314.08
                     2071-01-01       296.45       320.44       370.02
          -88.961884 1981-01-01       242.72       246.17       256.14
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01      1252.60      1315.04      1396.52
          -74.045807 1981-01-01      1189.65      1231.08      1246.81
                     2011-01-01      1202.68      1257.82      1306.37
                     2041-01-01      1213.78      1263.58      1321.78
                     2071-01-01      1245.08      1309.23      1389.14

[304824 rows x 3 columns]

## Eliminate 2011, Pivot Table and Create new columns 

In [15]:
Region1i85 = df85C.reset_index()
print (Region1i85)
Region1i285 = Region1i85[Region1i85.time.dt.year!= 2011]
print ("2011 eliminated")
df85f = Region1i285.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df85f.columns = [year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df85f

              lat        lon       time  prcptot_p10  prcptot_p50  prcptot_p90
0       66.623306 -89.045212 1981-01-01       245.04       248.60       258.56
1       66.623306 -89.045212 2011-01-01       260.38       266.67       274.81
2       66.623306 -89.045212 2041-01-01       276.36       296.33       314.08
3       66.623306 -89.045212 2071-01-01       296.45       320.44       370.02
4       66.623306 -88.961884 1981-01-01       242.72       246.17       256.14
...           ...        ...        ...          ...          ...          ...
304819  40.041039 -74.129143 2071-01-01      1252.60      1315.04      1396.52
304820  40.041039 -74.045807 1981-01-01      1189.65      1231.08      1246.81
304821  40.041039 -74.045807 2011-01-01      1202.68      1257.82      1306.37
304822  40.041039 -74.045807 2041-01-01      1213.78      1263.58      1321.78
304823  40.041039 -74.045807 2071-01-01      1245.08      1309.23      1389.14

[304824 rows x 6 columns]
2011 eliminated
DF pivote

hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212          973.74           977.01          1011.54   
          -88.961884          981.64           986.49          1019.84   
          -88.878548          974.04           978.58          1007.33   
          -88.795219          968.98           970.92          1000.87   
          -88.711891          971.74           975.16          1006.89   
...                              ...              ...              ...   
66.623306 -61.629639          492.61           560.37           603.77   
          -61.546310          502.43           572.46           617.11   
          -61.462978          497.35           564.91           609.59   
          -61.379650          492.91           560.57           605.55   
          -61.296322          465.43           528.44           572.22   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212         1010.56          1065.25          1081.71   
          -88.961884         1018.54          1071.99          1090.87   
          -88.878548         1011.78          1062.99          1083.85   
          -88.795219         1006.65          1056.69          1077.86   
          -88.711891         1008.74          1057.93          1080.27   
...                              ...              ...              ...   
66.623306 -61.629639          502.15           578.00           628.07   
          -61.546310          511.75           589.98           639.49   
          -61.462978          505.60           583.38           632.07   
          -61.379650          501.67           579.47           625.71   
          -61.296322          473.28           547.43           589.68   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212         1032.23          1098.97          1141.69  
          -88.961884         1041.09          1107.99          1152.78  
          -88.878548         1032.51          1101.09          1145.02  
          -88.795219         1026.73          1095.49          1137.33  
          -88.711891         1031.12          1095.82          1139.60  
...                              ...              ...              ...  
66.623306 -61.629639          520.73           627.34           723.44  
          -61.546310          530.46           640.81           738.68  
          -61.462978          524.30           631.76           728.02  
          -61.379650          520.04           627.35           721.71  
          -61.296322          490.46           591.94           677.40  

[76206 rows x 9 columns]

# Merge df45 and 85

In [16]:
dftg = pd.merge(df45f, df85f, on=["lat","lon"])
dftg.head()

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212          973.83          1005.69          1012.57   
          -88.961884          982.36          1014.96          1021.90   
          -88.878548          974.80          1008.30          1013.75   
          -88.795219          970.07          1003.25          1007.35   
          -88.711891          970.61          1005.58          1012.53   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212     1002.402344      1039.936279      1032.830688   
          -88.961884     1010.011169      1048.570557      1043.749878   
          -88.878548     1001.046082      1040.501587      1035.856567   
          -88.795219      993.866211      1037.199707      1028.240967   
          -88.711891      995.498840      1040.649048      1029.445435   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                            
40.041039 -89.045212     1022.678040      1117.536621      1113.804077   
          -88.961884     1031.998901      1128.370361      1121.904419   
          -88.878548     1024.193115      1120.681030      1113.541504   
          -88.795219     1018.400696      1113.942505      1106.774780   
          -88.711891     1021.307312      1116.597168      1106.486206   

                      hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212          973.74           977.01          1011.54   
          -88.961884          981.64           986.49          1019.84   
          -88.878548          974.04           978.58          1007.33   
          -88.795219          968.98           970.92          1000.87   
          -88.711891          971.74           975.16          1006.89   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212         1010.56          1065.25          1081.71   
          -88.961884         1018.54          1071.99          1090.87   
          -88.878548         1011.78          1062.99          1083.85   
          -88.795219         1006.65          1056.69          1077.86   
          -88.711891         1008.74          1057.93          1080.27   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212         1032.23          1098.97          1141.69  
          -88.961884         1041.09          1107.99          1152.78  
          -88.878548         1032.51          1101.09          1145.02  
          -88.795219         1026.73          1095.49          1137.33  
          -88.711891         1031.12          1095.82          1139.60

# Extract hist

In [17]:
dftg1 = dftg.drop(columns=["hist_rcp45_p10","hist_rcp45_p50", "hist_rcp45_p90", "hist_rcp85_p10", "hist_rcp85_p50", "hist_rcp85_p90"])
dftgh = dftg[["hist_rcp45_p50", "hist_rcp85_p50"]]

In [18]:
dftg1

t2050_rcp45_p10  t2080_rcp45_p10  t2050_rcp45_p50  \
lat       lon                                                             
40.041039 -89.045212          1005.69          1012.57      1039.936279   
          -88.961884          1014.96          1021.90      1048.570557   
          -88.878548          1008.30          1013.75      1040.501587   
          -88.795219          1003.25          1007.35      1037.199707   
          -88.711891          1005.58          1012.53      1040.649048   
...                               ...              ...              ...   
66.623306 -61.629639           537.15           538.33       551.150146   
          -61.546310           547.88           549.54       561.671448   
          -61.462978           541.19           542.73       555.124084   
          -61.379650           537.15           538.22       551.254639   
          -61.296322           506.54           506.88       520.979736   

                      t2080_rcp45_p50  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                             
40.041039 -89.045212      1032.830688      1117.536621      1113.804077   
          -88.961884      1043.749878      1128.370361      1121.904419   
          -88.878548      1035.856567      1120.681030      1113.541504   
          -88.795219      1028.240967      1113.942505      1106.774780   
          -88.711891      1029.445435      1116.597168      1106.486206   
...                               ...              ...              ...   
66.623306 -61.629639       564.163818       592.527039       630.359619   
          -61.546310       574.705933       603.849792       642.918396   
          -61.462978       567.502686       595.150452       635.752258   
          -61.379650       564.786560       590.644897       630.589417   
          -61.296322       533.535217       556.210083       595.162781   

                      t2050_rcp85_p10  t2080_rcp85_p10  t2050_rcp85_p50  \
lat       lon                                                             
40.041039 -89.045212           977.01          1011.54          1065.25   
          -88.961884           986.49          1019.84          1071.99   
          -88.878548           978.58          1007.33          1062.99   
          -88.795219           970.92          1000.87          1056.69   
          -88.711891           975.16          1006.89          1057.93   
...                               ...              ...              ...   
66.623306 -61.629639           560.37           603.77           578.00   
          -61.546310           572.46           617.11           589.98   
          -61.462978           564.91           609.59           583.38   
          -61.379650           560.57           605.55           579.47   
          -61.296322           528.44           572.22           547.43   

                      t2080_rcp85_p50  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                            
40.041039 -89.045212          1081.71          1098.97          1141.69  
          -88.961884          1090.87          1107.99          1152.78  
          -88.878548          1083.85          1101.09          1145.02  
          -88.795219          1077.86          1095.49          1137.33  
          -88.711891          1080.27          1095.82          1139.60  
...                               ...              ...              ...  
66.623306 -61.629639           628.07           627.34           723.44  
          -61.546310           639.49           640.81           738.68  
          -61.462978           632.07           631.76           728.02  
          -61.379650           625.71           627.35           721.71  
          -61.296322           589.68           591.94           677.40  

[76206 rows x 12 columns]

# Merge both hist_50 and get the mean

In [19]:
dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)
dftgh2 = dftgh.reset_index()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)



In [20]:
dftgh3 = dftgh2[["lat", "lon", "hist_p50"]]
dftgh3

,lat,lon,hist_p50
0,40.041039,-89.045212,1006.48
1,40.041039,-88.961884,1014.28
2,40.041039,-88.878548,1006.41
3,40.041039,-88.795219,1000.26
4,40.041039,-88.711891,1002.12
...,...,...,...
76201,66.623306,-61.629639,502.30
76202,66.623306,-61.546310,511.93
76203,66.623306,-61.462978,505.48
76204,66.623306,-61.379650,501.45


# Merge means with the 45 and 85 DF

In [21]:
dftg_all = pd.merge(dftgh3, dftg1, on=["lat","lon"])

In [22]:
dftg_all

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,1006.48,1005.69,1012.57,1039.936279,1032.830688,1117.536621,1113.804077,977.01,1011.54,1065.25,1081.71,1098.97,1141.69
1,40.041039,-88.961884,1014.28,1014.96,1021.90,1048.570557,1043.749878,1128.370361,1121.904419,986.49,1019.84,1071.99,1090.87,1107.99,1152.78
2,40.041039,-88.878548,1006.41,1008.30,1013.75,1040.501587,1035.856567,1120.681030,1113.541504,978.58,1007.33,1062.99,1083.85,1101.09,1145.02
3,40.041039,-88.795219,1000.26,1003.25,1007.35,1037.199707,1028.240967,1113.942505,1106.774780,970.92,1000.87,1056.69,1077.86,1095.49,1137.33
4,40.041039,-88.711891,1002.12,1005.58,1012.53,1040.649048,1029.445435,1116.597168,1106.486206,975.16,1006.89,1057.93,1080.27,1095.82,1139.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,502.30,537.15,538.33,551.150146,564.163818,592.527039,630.359619,560.37,603.77,578.00,628.07,627.34,723.44
76202,66.623306,-61.546310,511.93,547.88,549.54,561.671448,574.705933,603.849792,642.918396,572.46,617.11,589.98,639.49,640.81,738.68
76203,66.623306,-61.462978,505.48,541.19,542.73,555.124084,567.502686,595.150452,635.752258,564.91,609.59,583.38,632.07,631.76,728.02
76204,66.623306,-61.379650,501.45,537.15,538.22,551.254639,564.786560,590.644897,630.589417,560.57,605.55,579.47,625.71,627.35,721.71


In [24]:
dftg_all.to_feather("prcptot_annual.feather")

In [2]:
dftg_all = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/prcptot_annual.feather")

In [25]:
dftg_all 

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,1006.48,1005.69,1012.57,1039.936279,1032.830688,1117.536621,1113.804077,977.01,1011.54,1065.25,1081.71,1098.97,1141.69
1,40.041039,-88.961884,1014.28,1014.96,1021.90,1048.570557,1043.749878,1128.370361,1121.904419,986.49,1019.84,1071.99,1090.87,1107.99,1152.78
2,40.041039,-88.878548,1006.41,1008.30,1013.75,1040.501587,1035.856567,1120.681030,1113.541504,978.58,1007.33,1062.99,1083.85,1101.09,1145.02
3,40.041039,-88.795219,1000.26,1003.25,1007.35,1037.199707,1028.240967,1113.942505,1106.774780,970.92,1000.87,1056.69,1077.86,1095.49,1137.33
4,40.041039,-88.711891,1002.12,1005.58,1012.53,1040.649048,1029.445435,1116.597168,1106.486206,975.16,1006.89,1057.93,1080.27,1095.82,1139.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,502.30,537.15,538.33,551.150146,564.163818,592.527039,630.359619,560.37,603.77,578.00,628.07,627.34,723.44
76202,66.623306,-61.546310,511.93,547.88,549.54,561.671448,574.705933,603.849792,642.918396,572.46,617.11,589.98,639.49,640.81,738.68
76203,66.623306,-61.462978,505.48,541.19,542.73,555.124084,567.502686,595.150452,635.752258,564.91,609.59,583.38,632.07,631.76,728.02
76204,66.623306,-61.379650,501.45,537.15,538.22,551.254639,564.786560,590.644897,630.589417,560.57,605.55,579.47,625.71,627.35,721.71


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regioins: column name, short name

In [26]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
              "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"), 
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [27]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [28]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [30]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+replace_all(gdf[name].iloc[0], d)+"_prcptot_annual.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_prcptot_annual.json", driver="GeoJSON")

DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
0

ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Erabliere a caryer cordiforme_prcptot_annual.json: No such file or directory


Merged with polygons
Érablière à caryer cordiforme
Érablière à tilleul de l'Est
Érablière à bouleau jaune de l'Est
Érablière à tilleul de l'Ouest
Érablière à bouleau jaune de l'Ouest
Sapinière à bouleau jaune de l'Ouest
Sapinière à bouleau jaune de l'Est
Sapinière à bouleau blanc de l'Est
Golfe du Saint-Laurent
Sapinière à bouleau blanc de l'Ouest
Pessière à mousses de l'Ouest
Pessière à mousses de l'Est
Érablière à caryer cordiforme
Érablière à tilleul de l'Est
Érablière à bouleau jaune de l'Est
Érablière à tilleul de l'Ouest
Érablière à bouleau jaune de l'Ouest
Sapinière à bouleau jaune de l'Ouest
Sapinière à bouleau jaune de l'Est
Sapinière à bouleau blanc de l'Est


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Golfe du Saint-Laurent_prcptot_annual.json: No such file or directory


Golfe du Saint-Laurent
Sapinière à bouleau blanc de l'Ouest
Pessière à mousses de l'Ouest
Pessière à mousses de l'Est
Secteurs_Operations_Regionales
Merged with polygons
Secteur métropolitain et sud
Sud-Ouest
Centre du Québec
Nord-ouest
Sud-est
Nord-est
Secteur métropolitain et sud
Sud-Ouest
Centre du Québec
Nord-ouest
Sud-est
Nord-est
SREG_ECO
Merged with polygons
Plaine du bas Outaouais et de l'archipel de Montréal
Coteaux de l'Estrie
Collines du Mont-Mégantic
Coteaux de la rivière Chaudière
Plaine du Saint-Laurent
Collines de la basse Gatineau
Collines de Saint-Jérôme-Grand-Mère
Hautes collines du lac Simon
Collines du lac Dumont
Hautes collines de Val-David-Lac-Mékinac
Coteaux du lac Etchemin
Collines du lac Saint-Patrice
Massif du Mont-Tremblant
Collines du réservoir Kiamika
Collines du lac Kipawa
Collines du lac Notawassi
Collines de la rivière Vermillon
Coteaux du réservoir Dozois
Hautes collines de Saint-Tite-des-Caps
Hautes collines du lac Édouard
Collines et coteaux du lac Po

...

Coteaux de la rivière à la Croix et du lac au Griffon
Coteaux du lac Caopocho
Collines du lac Fonteneau


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Massif des monts Groulx_prcptot_annual.json: No such file or directory


Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_prcptot_annual.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet
territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
